# evaluate performance along various axes of sentence complexity

## data

In [1]:
print("start")
import sys
import os

sys.path.append("./enigma-transformed/src")
sys.path.append("./src")
sys.path.append("../src")
sys.path.append("../../src")

start


In [4]:
available_eval_columns = [
    "unigram_js_divergence",  # 17602 old, 18204
    "gpt2_tokens_per_char",  #  17659 old, 18206
    "gpt2_perplexity",  # 17604 old, 18207
    "bigram_js_divergence",  # 17657, 18209
    "depth_of_parse_tree", #17687
    "named_entities", #17688
    "pos_js_divergence", #17689
    "pos_bigram_js_divergence" #17690
]
eval_column = available_eval_columns[3]
print("eval_column:", eval_column)
dataset_max_len = 200

eval_column: unigram_js_divergence


In [12]:
import pandas as pd

data_path = f"news.2013.en.trainlen.{dataset_max_len}.merged.csv"
data = pd.read_csv(data_path)
# print len
print("len:", len(data))
# filter out rows which 'lang' != True
data = data[data['lang'] == True]
print("len:", len(data))
# weird is false
data = data[data['weird'] == False]
# print len
print("len:", len(data))
print("data loaded")
# sort by eval column
data.sort_values(eval_column, inplace=True)
print("data sorted")

# # print top 100 in original_text column
# print("top 100 in original_text column:")
# print(data['original_text'][:100])
# # bottom 100 in original_text column
# print("bottom 100 in original_text column:")
# print(data['original_text'][-100:])


len: 4888014
len: 4740817
len: 4726966
data loaded
data sorted
top 100 in original_text column:
3657034    The latter would remain safely in the realm of...
66317      Ostensibly - particularly when it came to the ...
681907     The report, commissioned by the anti-mining gr...
4534661    IRAN held two sets of talks on Wednesday aimed...
1218458    It was overwhelming - even after repeated eati...
                                 ...                        
3595778    The Missouri Supreme Court temporarily suspend...
3488527    Mr Shaw's decision to quit the Liberal Party o...
2245300    At the end of the week, the Moncler Gamme Roug...
632568     The transport firm revealed it has set aside m...
3916704    Dr Ali pointed to US President Barack Obama's ...
Name: original_text, Length: 100, dtype: object
bottom 100 in original_text column:
265842     Montenegro: Mladen Bozovic; Savo Pavicevic, St...
4130722    To join the Viking Cruises communities online,...
588808     Shakhter Karagan

: 

In [ ]:
# print the first 10 rows
print(f"{data.head(10)=}")
# last 10 rows
print(f"{data.tail(10)=}")

rows = {
    0.1: data.iloc[len(data) // 1000 * 999 - 1000 : len(data) // 1000 * 999],
    1: data.iloc[len(data) // 100 * 99 - 1000 : len(data) // 100 * 99],
    5: data.iloc[len(data) // 100 * 95 - 1000 : len(data) // 100 * 95],
    10: data.iloc[len(data) // 100 * 90 - 1000 : len(data) // 100 * 90],
    15: data.iloc[len(data) // 100 * 85 - 1000 : len(data) // 100 * 85],
    20: data.iloc[len(data) // 100 * 80 - 1000 : len(data) // 100 * 80],
    25: data.iloc[len(data) // 100 * 75 - 1000 : len(data) // 100 * 75],
    30: data.iloc[len(data) // 100 * 70 - 1000 : len(data) // 100 * 70],
    35: data.iloc[len(data) // 100 * 65 - 1000 : len(data) // 100 * 65],
    40: data.iloc[len(data) // 100 * 60 - 1000 : len(data) // 100 * 60],
    45: data.iloc[len(data) // 100 * 55 - 1000 : len(data) // 100 * 55],
    50: data.iloc[len(data) // 100 * 50 - 1000 : len(data) // 100 * 50],
    55: data.iloc[len(data) // 100 * 45 - 1000 : len(data) // 100 * 45],
    60: data.iloc[len(data) // 100 * 40 - 1000 : len(data) // 100 * 40],
    65: data.iloc[len(data) // 100 * 35 - 1000 : len(data) // 100 * 35],
    70: data.iloc[len(data) // 100 * 30 - 1000 : len(data) // 100 * 30],
    75: data.iloc[len(data) // 100 * 25 - 1000 : len(data) // 100 * 25],
    80: data.iloc[len(data) // 100 * 20 - 1000 : len(data) // 100 * 20],
    85: data.iloc[len(data) // 100 * 15 - 1000 : len(data) // 100 * 15],
    90: data.iloc[len(data) // 100 * 10 - 1000 : len(data) // 100 * 10],
    95: data.iloc[len(data) // 100 * 5 - 1000 : len(data) // 100 * 5],
    99: data.iloc[len(data) // 100 * 1 - 1000 : len(data) // 100 * 1],
    99.9: data.iloc[len(data) // 1000 * 1 - 1000 : len(data) // 1000 * 1],
}
print("percentile rows split")

## load model

In [ ]:
from transformers import ByT5Tokenizer, T5ForConditionalGeneration
from src.utils import levensthein_distance, print_avg_median_mode_error
from transformers import pipeline, logging
import torch

logging.set_verbosity(logging.ERROR)


tokenizer = ByT5Tokenizer()

from src.ByT5Dataset import ByT5CaesarRandomDataset, ByT5ConstEnigmaDataset

model = T5ForConditionalGeneration.from_pretrained("./logs/slurm_17510/model")
dataset_class = ByT5ConstEnigmaDataset  # for 17510 model
# dataset_class = ByT5CaesarRandomDataset # for 16677 model
rows_datasets = {i: dataset_class(list(rows[i].text), dataset_max_len) for i in rows}

print("rows_datasets created")

averages, medians, modes = {}, {}, {}
raw_data = {}
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
for i, test in rows_datasets.items():
    print(f"#############################################")
    print(f"Testing {i}th percentile")
    error_counts = []
    translate = pipeline("translation", model=model, tokenizer=tokenizer, device=device)
    for index in range(len(test)):
        generated = translate(
            test[index]["input_text"], max_length=(dataset_max_len + 1) * 2
        )[0]["translation_text"]
        error_counts.append(levensthein_distance(generated, test[index]["output_text"]))
        if error_counts[-1] > 0:
            print(f"Example {index}, error count {error_counts[-1]}")
            print("In :", test[index]["input_text"])
            print("Gen:", generated)
            expected = test[index]["output_text"]
            print("Exp:", expected)
        else:
            print(f"Example {index} OK")
        print("-----------------------")

    avg, med, mode = print_avg_median_mode_error(error_counts)
    averages[i] = avg
    medians[i] = med
    modes[i] = mode
    raw_data[i] = error_counts

print("Averages:", averages)
print("Medians:", medians)
print("Modes:", modes)
print("Raw data:", raw_data)